### 2.2 Tokenizing text

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [3]:
with open("the-verdict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()

In [4]:
raw_text

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)\n\n"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it\'s going to send the value of my picture \'way up; but I don\'t think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing\'s lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn\'s "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?\n\nWell!--even 

In [5]:
len(raw_text)

20479

In [6]:
import re

text = 'Hello, world. This, is a test.'
result = re.split(r'(\s)', text)
result

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']

In [7]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [8]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [9]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [10]:
result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
result = [item.strip() for item in result if item.strip()]
preprocessed = result

In [11]:
len(preprocessed)

4690

In [12]:
preprocessed[:10]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius']

### 2.3 Converting tokens into token IDs

In [13]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
vocab_size

1130

In [14]:
vocab = {
    token: integer for integer, token in enumerate(all_words)
}
# vocab

In [15]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [16]:
tokenizer = SimpleTokenizerV1(vocab)

In [17]:

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [18]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [19]:
tokenizer.decode(tokenizer.encode('is the in the.'))

'is the in the.'

### 2.4 Adding special context tokens

In [20]:
text = "Hello, do you like tea? Is this a test?"
try:
    tokenizer.encode(text)
except Exception as e:
    import traceback
    traceback.print_exc()     

Traceback (most recent call last):
  File "/var/folders/3d/l96nwm_90mb1gc_pf7zvv59m0000gn/T/ipykernel_4479/2488907652.py", line 3, in <module>
    tokenizer.encode(text)
    ~~~~~~~~~~~~~~~~^^^^^^
  File "/var/folders/3d/l96nwm_90mb1gc_pf7zvv59m0000gn/T/ipykernel_4479/2118097954.py", line 12, in encode
    ids = [self.str_to_int[s] for s in preprocessed]
           ~~~~~~~~~~~~~~~^^^
KeyError: 'Hello'


In [21]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend([
    '<|endoftext|>', "<|unk|>"
])
vocab = {
    token: integer for integer, token in enumerate(all_tokens)
}

In [22]:
len(vocab.items())

1132

In [23]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [24]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [25]:
text = "Hello, do you like tea? Is this a test?"
tokenizer = SimpleTokenizerV2(vocab)
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1131, 999, 115, 1131, 10]

In [26]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|unk|> this a <|unk|>?'

### 2.5 BytePair encoding

In [27]:
import tiktoken

In [28]:
tiktoken.__version__

'0.11.0'

In [29]:
tokenizer = tiktoken.get_encoding("gpt2")

In [30]:
tokenizer.encode('Hello world')

[15496, 995]

In [31]:
tokenizer.decode(tokenizer.encode('Hello world'))

'Hello world'

In [32]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [33]:
tokenizer.decode(integers)

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'

### 2.6 Data sampling with a sliding window

In [34]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [35]:
enc_sample = enc_text[50:]

In [36]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [37]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [38]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [39]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.8.0


In [40]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [41]:
txt = 'Rahul is the best in the world, he is going to be the CEO of skartner'
max_length = 5
stride = 1

dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

print(tokenizer.decode(dataset[0][0].tolist()))
print(tokenizer.decode(dataset[0][1].tolist()))

print(tokenizer.decode(dataset[1][0].tolist()))
print(tokenizer.decode(dataset[1][1].tolist()))

Rahul is the best
ul is the best in
ul is the best in
 is the best in the


In [42]:
txt = 'Rahul is the best in the world, he is going to be the CEO of skartner'
max_length = 5
stride = 5

dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

print(tokenizer.decode(dataset[0][0].tolist()))
print(tokenizer.decode(dataset[0][1].tolist()))

print(tokenizer.decode(dataset[1][0].tolist()))
print(tokenizer.decode(dataset[1][1].tolist()))

Rahul is the best
ul is the best in
 in the world, he
 the world, he is


In [43]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [44]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [45]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=4, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [46]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]


In [47]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


### 2.7 Creating token embeddings

In [51]:
input_ids = torch.tensor([2, 3, 5, 1])

In [52]:
tokenizer.n_vocab

50257

In [54]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [55]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [56]:
embedding_layer(torch.tensor([3]))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)

In [59]:
embedding_layer(input_ids)

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)

In [60]:
input_ids

tensor([2, 3, 5, 1])

In [61]:
vocab_size = 6
output_dim = 4

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [64]:
embedding_layer(torch.tensor([1]))

tensor([[ 0.3486,  0.6603, -0.2196, -0.3792]], grad_fn=<EmbeddingBackward0>)

### 2.8 Encoding word positions

In [66]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [67]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [68]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [69]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings.shape

torch.Size([8, 4, 256])

In [87]:
token_embeddings[0]

tensor([[ 0.6914,  0.5580,  0.6737,  ...,  0.9702, -0.0817, -1.5842],
        [-0.5060, -0.7649, -0.7381,  ...,  0.0335, -0.2869,  1.0283],
        [ 0.0682, -0.3329, -1.3527,  ..., -1.5163,  0.5293, -0.6115],
        [-0.1771, -0.0126, -0.2949,  ...,  2.1229, -0.4288,  0.0852]],
       grad_fn=<SelectBackward0>)

In [72]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [75]:
pos_embedding_layer.weight

Parameter containing:
tensor([[-2.3757,  0.3616,  1.6211,  ..., -0.6813, -1.1042, -0.5608],
        [ 1.1531,  0.0796,  0.5718,  ...,  1.3845, -1.1687,  2.1513],
        [ 0.2339, -0.5470,  0.7052,  ...,  0.1407, -1.9278, -1.4534],
        [-0.2963, -0.2133,  0.5776,  ..., -0.0176,  0.8153, -1.4964]],
       requires_grad=True)

In [73]:
torch.arange(max_length)

tensor([0, 1, 2, 3])

In [76]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings)

tensor([[-2.3757,  0.3616,  1.6211,  ..., -0.6813, -1.1042, -0.5608],
        [ 1.1531,  0.0796,  0.5718,  ...,  1.3845, -1.1687,  2.1513],
        [ 0.2339, -0.5470,  0.7052,  ...,  0.1407, -1.9278, -1.4534],
        [-0.2963, -0.2133,  0.5776,  ..., -0.0176,  0.8153, -1.4964]],
       grad_fn=<EmbeddingBackward0>)


In [77]:
pos_embeddings.shape

torch.Size([4, 256])

In [78]:
token_embeddings.shape

torch.Size([8, 4, 256])

In [79]:
token_embeddings[0] + pos_embeddings

tensor([[-1.6843,  0.9196,  2.2948,  ...,  0.2890, -1.1859, -2.1449],
        [ 0.6471, -0.6854, -0.1663,  ...,  1.4180, -1.4556,  3.1796],
        [ 0.3021, -0.8799, -0.6475,  ..., -1.3756, -1.3985, -2.0649],
        [-0.4734, -0.2259,  0.2827,  ...,  2.1053,  0.3865, -1.4111]],
       grad_fn=<AddBackward0>)

In [81]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
